# Deploying a model with live microphone input

In this example we will walk through loading a model exported from NengoEdge,
and deploying that model in a simple application that detects
keywords from live microphone input.

## Installation

First, install NengoEdge tools
using [these instructions](https://www.nengo.ai/nengo-edge/developers.html).

For this example, you will also need to install:

- [sounddevice](https://python-sounddevice.readthedocs.io/en/latest/)
- [Jupyter Notebook or JupyterLab](https://jupyter.org/)

### Conda/Mamba

```bash
conda activate nengo-edge
conda install python-sounddevice notebook
```

### pip

On Ubuntu/Debian, first:

```bash
sudo apt install libportaudio2
```

On all platforms, including Ubuntu/Debian:

```bash
pip install sounddevice notebook
```

In [ ]:
# Ensure dependencies are installed
try:
    import sounddevice

    assert sounddevice
except ImportError:
    print("sounddevice is not installed. See above for instructions.")
    raise

## Train a model in NengoEdge

The first step is to train a model in NengoEdge. See [this blog
post](https://appliedbrainresearch.com/blog/fast-keyword-detection-with-lmus-on-gpu)
for a detailed walkthrough on how to train such a model.

You can use whatever dataset you like, but make sure to match the parameters below
to your training data.

In [ ]:
# Keyword labels for model outputs
labels = [
    "<silence>",
    "<unknown>",
    "yes",
    "no",
    "up",
    "down",
    "left",
    "right",
    "on",
    "off",
    "stop",
    "go",
]

# Sample rate of audio files in the dataset
sample_rate = 16000

After training your model in NengoEdge, download the "TFLite" artifact
from the Deployment page.

Make sure to set the string passed to `TFLiteRunner` to
the directory containing the extracted deployment files.

In [ ]:
import numpy as np
import sounddevice as sd  # pylint: disable=ungrouped-imports

from nengo_edge import TFLiteRunner

runner = TFLiteRunner(".")

Next we need to select the microphone input device:

In [ ]:
print(f"Available devices:\n{sd.query_devices()}")
mic_device_ix = int(input("Index of input device to listen to: "))

And finally we can run the model using that device. As you speak into your microphone,
it will print out the keywords detected by the model.

Note that the particular input method below is not a requirement. The important part is
that we call `runner.run` in a loop, feeding it the new audio samples each time. This
can be adapted to whatever approach makes the most sense for your application.

In [ ]:
inference_rate = 0.2  # How often to do a classification, in seconds
block_size = int(sample_rate * inference_rate)


def callback(data, frames, time, status):
    # Call model periodically as new data comes in
    outputs = runner.run(data.T)
    label = labels[np.argmax(outputs)]
    if label != "<silence>":
        print(label)


try:
    with sd.InputStream(
        device=mic_device_ix,
        samplerate=sample_rate,
        blocksize=block_size,
        latency="low",
        channels=1,
        dtype=np.float32,
        callback=callback,
    ):
        print("Press Enter to quit")
        input()
    print("Exiting")
except KeyboardInterrupt:
    print("Exiting")